## Initial Setup

The setup below shows in-code configuration. For most applications, we can also enable LLMObs simply by calling `ddtrace-run` with the appropriate env vars, [as seen here in our quickstart instructions](https://docs.datadoghq.com/tracing/llm_observability/quickstart/).

The code below requires you to have already created an `.env` file with several configuration variables as explained in the README.


In [1]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

**Note for enterprise customers using secrets:**

If you are using secrets, you can enable LLM Observability with more specific parameters as demonstrated below.

```python
LLMObs.enable(
  ml_app="<YOUR_ML_APP_NAME>",
  api_key="<YOUR_DATADOG_API_KEY>",
  site="<YOUR_DATADOG_SITE>",
  agentless_enabled=True,
)
```


## Tracing an LLM Call

An LLM Span represents a call to a model. In this simple example, we are asking Claude via Amazon Bedrock to summarize a provided text and identify a list of topics from the text.

Because we use Amazon Bedrock with Datadog's instrumentation, the call to the LLM is automatically traced:


In [2]:
import boto3
import json
import os

# Initialize Bedrock client
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # specify your region
)

def get_model_id(model_name="claude"):
    """
    Get the model ID based on the model name
    Default is Claude-2
    """
    model_mapping = {
        "claude": "anthropic.claude-v2",  # Using Claude-2 as default
        "claude-2": "anthropic.claude-v2",
        "claude-instant": "anthropic.claude-instant-v1"
    }
    return model_mapping.get(model_name, model_mapping["claude"])


sys_prompt = """
Your task is to 
1. Summarize the given text at a 6th grade reading level in no more than 2 sentences.
2. Identify what topics the text belongs to that would allow you to categorize it in a school library.
Format your output strictly following this JSON convention:
{	
    "topics": <[insert array of topics here]>
    "summary": <insert summary here>
}
	"""


def summarize(text, model_name="claude", prompt=sys_prompt):
    # Construct the message in Claude's format
    message = f"{prompt}\n\nHuman: {text}\n\nAssistant: "
    
    try:
        response = bedrock.invoke_model(
            modelId=get_model_id(model_name),
            body=json.dumps({
                "prompt": message,
                "max_tokens_to_sample": 1000,  # Changed from max_tokens
                "temperature": 0,
                "anthropic_version": "bedrock-2023-05-31"
            })
        )
        
        # Parse the response
        response_body = json.loads(response.get('body').read())
        print(f"Response body: {response_body}")

        #return json.loads(response_body.get('completion'))
        

        completion_text = response_body.get('completion', '').strip()
        
        # Find the JSON part within the completion
        try:
            # Look for content between curly braces
            json_start = completion_text.find('{')
            json_end = completion_text.rfind('}') + 1
            if json_start >= 0 and json_end > 0:
                json_str = completion_text[json_start:json_end]
                return json.loads(json_str)
            else:
                raise ValueError("No JSON found in response")
        except json.JSONDecodeError:
            print("Failed to parse JSON from response")
            return None
        
    except Exception as e:
        print(f"Error calling Bedrock: {str(e)}")
        raise

In [3]:
text = """
ONE JANUARY day, thirty years ago, the little town of Hanover, anchored on a windy Nebraska tableland, was trying not to be blown away. A mist of fine snowflakes was curling and eddying about the cluster of low drab buildings huddled on the gray prairie, under a gray sky. The dwelling-houses were set about haphazard on the tough prairie sod; some of them looked as if they had been moved in overnight, and others as if they were straying off by themselves, headed straight for the open plain. None of them had any appearance of permanence, and the howling wind blew under them as well as over them. The main street was a deeply rutted road, now frozen hard, which ran from the squat red railway station and the grain "elevator" at the north end of the town to the lumber yard and the horse pond at the south end. On either side of this road straggled two uneven rows of wooden buildings; the general merchandise stores, the two banks, the drug store, the feed store, the saloon, the post-office. The board sidewalks were gray with trampled snow, but at two o'clock in the afternoon the shopkeepers, having come back from dinner, were keeping well behind their frosty windows.
"""

In [5]:
summarize(text)

Response body: {'type': 'completion', 'completion': ' Here is a 6th grade level summary of the text in 2 sentences:\n\n{\n    "topics": ["Geography", "History", "Small Towns"],\n    "summary": "Thirty years ago, the small prairie town of Hanover was trying not to be blown away by wind and snow, with its basic shops and houses spread out on the flat, treeless land."  \n}', 'stop_reason': 'stop_sequence', 'stop': '\n\nHuman:'}


{'topics': ['Geography', 'History', 'Small Towns'],
 'summary': 'Thirty years ago, the small prairie town of Hanover was trying not to be blown away by wind and snow, with its basic shops and houses spread out on the flat, treeless land.'}

failed to send, dropping 1 traces to intake at https://llmobs-intake.datadoghq.com/api/v2/llmobs after 5 retries, 1 additional messages skipped


## Viewing the trace in Datadog

Now, check out the [LLM Observability interface](https://app.datadoghq.com/llm) in Datadog. You should see a trace that describes the LLM call, including the system prompt, the user prompt, and the response.


#### Additional resources

- [List of all integrations supported by Datadog's LLM Observability product](https://docs.datadoghq.com/tracing/llm_observability/sdk/#llm-integrations)
- [Instructions for manually instrumenting an LLM Span](https://docs.datadoghq.com/llm_observability/setup/sdk/python/#llm-span)
